# Import

In [ ]:
%load_ext autoreload
%autoreload 2

import importlib
import numpy as np
import math
import itertools 
import pandas as pd
import numpy as np
import io
import msoffcrypto
from sklearn.metrics import r2_score
import openpyxl
import plotly.express as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st
import json
from math import factorial as fact
from IPython.display import Markdown, display
import toml
import sys, os
import collections
from types import SimpleNamespace
import copy
import matplotlib.patches as mpatch
%matplotlib inline 
%config Completer.use_jedi = True

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from utils import *

# Run

In [ ]:
file = {'SheetMapName' : 'map', 'uploaded_file' : "data.xlsx"}
file = {'SheetMapName' : 'map', 'uploaded_file' : None, 'DistFactor' : 0.1}
algo = load_data_brut(file)
# algo.dfmap

In [ ]:
algo.BusActif = True
algo.Npc = 0

In [ ]:
keydrop= ['Nvals',"confs","dfcapteur", "dfslot","dfline","indivs",
          "df",'dfmap','A0','DataCategorie', 'DictLine','DictPos','dist','durite',
        'duriteType','duriteVal','CombAll','Comb']
d = {k : v for k,v in vars(algo).items() if k not in keydrop}
s = pd.Series(d).rename('Val').astype(str)
s.index= s.index.astype(str)
s

In [ ]:
# generation de 10 indivs
df = indiv_init(algo, 10)

In [ ]:
# columns de l'algo genetique
col = ['Clist', 'CtoE', 'Econnect', 'Elist', 'Ecount', 'EtoP', 'Pconnect','Plist', 'Pcount']

In [ ]:
# mutation
row = df.iloc[0].copy()
dfMutation = pd.Series(Mutation(row, algo))[col]
dfresults  = pd.concat([row,dfMutation], axis = 1)
dfresults.columns = ['parent','enfant']
dfresults.loc[col]

In [ ]:
# crossover
dfparents = df[:2].copy()
dfcrossover = pd.DataFrame(AG_CrossOver(dfparents, algo))
dfresults  = pd.concat([dfparents.T,dfcrossover.T ], axis = 1)
dfresults.columns = ['parent1','parent2','enfant1','enfant2']
dfresults.loc[col]

In [ ]:
# Plot
row = df.iloc[0]
ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
SelectSlot = []
List_EtoC = row.List_EtoC
List_PtoE = row.List_PtoE
for n in range(3):
    SelectSlot+= ['{}{}'.format(Elems[n],i) for i in row[ElemsList[n]]]
SelectLine = row.Name
if row.Option == 'Bus' :   SelectLine = row.BusName                
fig = new_plot(algo, SelectLine, SelectSlot)

In [ ]:
# Plot
SelectLine = algo.DictLine.keys()
SelectSlot = algo.DictPos.keys()

fig = new_plot(algo, SelectLine, SelectSlot)

## new formule vrac

In [ ]:
Sindiv = pd.Series(indiv)

In [ ]:
df.columns
indiv = algo.indivs[0]
Calcul_Debit(algo ,indiv, Split = algo.Split)

In [ ]:
debitinput = {'p': 0, 'e': 0, 'ClistG': [0, 1, 3], 'pt': 'Pa'}
debit(algo,indiv,debitinput, grouped = True, split = False)

In [ ]:
pompe,ev,ClistG,pt = debitinput.values()
d_EtoC_list = np.array([algo.dist['E{}-C{}'.format(ev,c)] for c in ClistG])
d_PtoE      = indiv['dist_Connect']['P{}-E{}'.format(pompe,ev)]
d_EtoC_list, d_PtoE
d_EtoC_list = [1,1,1]
d_PtoE = 1
p = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
p
grouped = True
split = False
cE0 = 7.64e-04
coef_E = 0 if split else cE0

coef_C  = 0.036
coef_C  = [algo.Nvals[i] for i in ClistG]
coef_C  = np.array(coef_C)
coef_d_EtoC  = 2.35e-04
coef_d_EtoC = np.array([algo.duriteVal['E{}-C{}'.format(ev,c)] for c in ClistG])
# coef_d_PtoE = algo.duriteVal['P{}-E{}'.format(pompe,ev)] 
coef_d_PtoE = indiv['duriteVal']['P{}-E{}'.format(pompe,ev)] 
a,b,c = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
coef_E ,d_EtoC_list,  coef_d_EtoC, coef_C
A

In [ ]:
Q = 40
A*10
a * Q**2 + b*Q +c - coef_d_PtoE*Q**2
a * Q**2 + b*Q +c - coef_d_PtoE


In [ ]:
coef_E
Ncg=  6
Ncm = 12
d_EtoC_list = np.ones(Ncm) * 1
coef_d_EtoC = np.ones(Ncm) * 0.000235
coef_C = np.ones(Ncm) * 0.036
a,b,c = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
Q0 = np.arange(0.1,80,0.1)
Qx = np.zeros(Q0.shape)
Qi = np.zeros((12,len(Q0)))
Nev = int(Ncm/6)
for i in range(Nev):
    Q = Q0 - Qx
    # Q.shape, Qx.sum()    
    # Q = Q0.copy()
    # Q.shape
    G = a * Q**2 + b*Q +c - coef_d_PtoE*(i+1)*Q**2
    G[G <0] = 0
    # print(G)
    Qi[i*Ncg:Ncg*(i+1),:] = np.sqrt(G / A[i*Ncg:Ncg*(i+1),np.newaxis]).round(3)
    Qx = Qi[0:Ncg*(i+1),:].sum(0)
    # Qx
    # pd.DataFrame(Qi)
    # Qx.shape
    f = plt.plot(Qx)
    # Qi.shape
dfr = pd.DataFrame(Qi).T
dfr.index = Q0
idx = np.searchsorted(Q0 - Qi.sum(0), 0)
np.isclose(Qi.sum(0),Q0,atol= 0.1).sum()
# np.isclose(Qi.sum(0),Q,atol= 0.1)
idx, Q[idx],Qi[:,idx], Qi.shape, G[idx]

In [ ]:
a,b,c = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
a,b,c

In [ ]:

Q = np.arange(0.1,80,0.1)
G = a * Q**2 + b*Q +c - coef_d_PtoE
Qi= np.sqrt(G / A[:,np.newaxis])
Q[np.isclose(Qi.sum(0),Q,atol= 0.1)]
Qi[:,np.isclose(Qi.sum(0),Q,atol= 0.1)]
idx = np.searchsorted(Q - Qi.sum(0), 0)
Q[idx],Qi[:,idx]
Qi.shape
# Q[np.isclose(Qi.sum(0),Q,atol= 0.1)]

In [ ]:
%timeit np.isclose(Qi.sum(0),Q,atol= 0.1)
%timeit np.searchsorted(Q - Qi.sum(0), 0.1)


In [ ]:
Qi.sum(0) - Q

In [ ]:
%%timeit
G = a * Q**2 + b*Q +c - coef_d_PtoE
Qi= np.sqrt(G / A[:,np.newaxis])


In [ ]:
p = np.array(p)

In [ ]:
%%timeit
Q = 40
Qs = []
for i in range(40):
    G = a * Q**2 + b*Q +c - coef_d_PtoE
    Qi= np.sqrt(G / A)
    Q = np.sum(Qi)
    Qs.append(Q)

In [ ]:

Q = 40
Qs = []
for i in range(40):
    G = a * Q**2 + b*Q +c - coef_d_PtoE
    Qi= np.sqrt(G / A)
    Q = np.sum(Qi)
    Qs.append(Q)
plt.plot(Qs)
Qs[-1]
# Qi


In [ ]:
indiv['dist_Connect']
A

In [ ]:
pd.Series(indiv)

# Test VRAC a trier 

In [ ]:
pompe,ev,ClistG,pt = debitinput.values()
d_EtoC_list = np.array([algo.dist['E{}-C{}'.format(ev,c)] for c in ClistG])
d_PtoE      = indiv['dist_Connect']['P{}-E{}'.format(pompe,ev)]
d_EtoC_list, d_PtoE
d_EtoC_list = [1,1,1]
d_PtoE = 1
p = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
p
grouped = True
split = False
cE0 = 7.64e-04
coef_E = 0 if split else cE0

coef_C  = 0.036
coef_C  = [algo.Nvals[i] for i in ClistG]
coef_C  = np.array(coef_C)
coef_d_EtoC  = 2.35e-04
coef_d_EtoC = np.array([algo.duriteVal['E{}-C{}'.format(ev,c)] for c in ClistG])
# coef_d_PtoE = algo.duriteVal['P{}-E{}'.format(pompe,ev)] 
coef_d_PtoE = indiv['duriteVal']['P{}-E{}'.format(pompe,ev)] 
a,b,c = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
coef_E ,d_EtoC_list,  coef_d_EtoC, coef_C
a,b,c
A

In [ ]:
Q = 2
# A*10
a * Q**2 + b*Q +c - coef_d_PtoE*Q**2
a * Q**2 + b*Q +c - coef_d_PtoE
coef_d_PtoE*Q**2

In [ ]:
coef_d_PtoE*2**2

In [ ]:
coef_d_PtoE

In [ ]:
Q = np.arange(0.1,80,0.1)
Ncg=  1
Ncm = 1
d_EtoC_list = np.ones(Ncm) * 1
coef_d_EtoC = np.ones(Ncm) * 0.000235
coef_C = np.ones(Ncm) * 0.036
i = 0
G = a * Q**2 + b*Q +c - coef_d_PtoE*(i+1)*Q**2
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
A = A[0]
G[G <0] = 0
Qi = np.sqrt(G / A).round(3)
idx = np.searchsorted(Q0 - Qi.sum(0), 0)
plt.plot(Q)
plt.plot(Qi)
idx = np.searchsorted(Q - Qi, 0)
idx, Q[idx],Qi[idx], Qi.shape, G[idx]

mask = np.isclose(Qi,Q,atol= 0.1)
Qi[mask]

In [ ]:
Q = np.arange(0.1,80,0.1)
Ncg=  1
Ncm = 1
d_EtoC_list = np.ones(Ncm) * 1
coef_d_EtoC = np.ones(Ncm) * 0.000235
coef_C = np.ones(Ncm) * 0.036
i = 0
Qi = [0,0]
G = a * Q**2 + b*Q +c - coef_d_PtoE*(i+1)*Q**2
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
A = A[0]
G[G <0] = 0
Q1 = np.sqrt(G / A).round(3)
G2 = G - coef_d_PtoE*(Q-Qi[0])**2
G2[G2 <0] = 0
Q2 = np.sqrt(G2 / A).round(3)

idx = np.searchsorted(Q0 - (Q1 + Q2), 0)
idx
# plt.plot(Q)
# plt.plot(Qi)
# np.vstack([Q1,Q2])
# Q1
# plt.plot(np.vstack([Q1,Q2]))
# idx = np.searchsorted(Q - Qi, 0)
# idx, Q[idx],Qi[idx], Qi.shape, G[idx]
Q1[idx], Q2[idx]
# mask = np.isclose(Qi,Q,atol= 0.1)
# Qi[mask]

In [ ]:
coef_d_PtoE

In [ ]:
coef_d_PtoE

In [ ]:
Ncg=  2
Ncm = 6

d_EtoC_list = np.ones(Ncm) * 1
coef_d_EtoC = np.ones(Ncm) * 0.000235
coef_C = np.ones(Ncm) * 0.036
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
A
Q0 = np.arange(0.1,80,0.1)
Qx = np.zeros(Q0.shape)
Qi = np.zeros((Ncm,len(Q0)))

G = a * Q0**2 + b*Q0 +c
# Qx = 0
Nev = int(Ncm/Ncg)
# coef_d_PtoE = np.ones(Nev) * 0.000235
coef_d_PtoE = np.array([1,2,4]) * 0.000235
for i in range(Nev):
#     print(i)
    G = G - coef_d_PtoE[i]*(Q0-Qx)**2
    G[G <0] = 0
    Qi[i*Ncg:Ncg*(i+1),:] = np.sqrt(G / A[i*Ncg:Ncg*(i+1),np.newaxis])
    Qx = Qi.sum(0)
f = plt.plot(Qx)
f = plt.plot(Q0)
dfr = pd.DataFrame(Qi).T
dfr.index = Q0
# dfr
idx = np.searchsorted(Q0 - Qx, -1)
# mask = np.isclose(Qi.sum(0),Q0,atol= 1)
# Qx[mask]

# np.isclose(Qi.sum(0),Q,atol= 0.1)
idx, Q[idx],Qx[idx], Qi[:,idx], Qi.shape, G[idx]

# Final loop 

In [ ]:
df.columns
indiv = algo.indivs[0]
Calcul_Debit(algo ,indiv, Split = algo.Split)

In [ ]:
debitinput = {'p': 0, 'e': 0, 'ClistG': [0, 1, 3], 'pt': 'Pa'}
debit(algo,indiv,debitinput, grouped = True, split = False)

In [ ]:
pompe,ev,ClistG,pt = debitinput.values()
d_EtoC_list = np.array([algo.dist['E{}-C{}'.format(ev,c)] for c in ClistG])
d_PtoE      = indiv['dist_Connect']['P{}-E{}'.format(pompe,ev)]
d_EtoC_list, d_PtoE
d_EtoC_list = [1,1,1]
d_PtoE = 1
p = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
p
grouped = True
split = False
cE0 = 7.64e-04
coef_E = 0 if split else cE0

coef_C  = 0.036
coef_C  = [algo.Nvals[i] for i in ClistG]
coef_C  = np.array(coef_C)
coef_d_EtoC  = 2.35e-04
coef_d_EtoC = np.array([algo.duriteVal['E{}-C{}'.format(ev,c)] for c in ClistG])
# coef_d_PtoE = algo.duriteVal['P{}-E{}'.format(pompe,ev)] 
coef_d_PtoE = indiv['duriteVal']['P{}-E{}'.format(pompe,ev)] 
a,b,c = [algo.DataCategorie['Pompe']['Values'][pt][i] for i in ['a','b','c']]
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 
coef_E ,d_EtoC_list,  coef_d_EtoC, coef_C
a,b,c
A

In [ ]:

Ncg=  6
Ncm = 12
cE0 = 7.64e-04
coef_E =  cE0
d_EtoC_list = np.ones(Ncm) * 1
coef_d_EtoC = np.ones(Ncm) * 0.000235
coef_C      = np.ones(Ncm) * 0.036
A = coef_E + d_EtoC_list * coef_d_EtoC + coef_C 

Q0 = np.arange(0.1,80,0.1)
Qx = np.zeros(Q0.shape)
Qi = np.zeros((Ncm,len(Q0)))

G = a * Q0**2 + b*Q0 +c
Nev = int(Ncm/Ncg)
coef_d_PtoE = np.ones(Nev) * 0.000235
for i in range(Nev):
    G = G - coef_d_PtoE[i]*(Q0-Qx)**2
    G[G <0] = 0
    Qi[i*Ncg:Ncg*(i+1),:] = np.sqrt(G / A[i*Ncg:Ncg*(i+1),np.newaxis])
    Qx = Qi.sum(0)

idx = np.searchsorted(Q0 - Qx, -0.9)

idx, Q0[idx],Qx[idx], Qi[:,idx], Qi.shape, G[idx]

In [ ]:
f = plt.plot(Qx)
f = plt.plot(Q0)